In [1]:
import pandas as pd
import itertools

In [2]:
prediction_dir = '../predictions/'
triplets_dir = '../data/triplets/'
drugbank_dir = '../data/drugbank/'

In [3]:
drug_name_map = pd.read_csv(drugbank_dir + 'drug_id_name_map.csv', sep=',', index_col=[0])
food_name_map = pd.read_csv(triplets_dir + 'food_name.tsv', sep='\t', index_col=[0])
food_compound_map = pd.read_csv(triplets_dir + 'compounds_names.tsv', sep='\t', index_col=[0])

In [4]:
drug_ids = drug_name_map.id
drug_names = drug_name_map.drug_name
drug_id_map_dict = dict(zip(drug_ids, drug_names))

food_ids = food_name_map.public_id
food_names = food_name_map.name
food_id_map_dict = dict(zip(food_ids, food_names))

compound_ids = food_compound_map.compound_id
compound_names = food_compound_map.name
compound_id_map_dict = dict(zip(compound_ids, compound_names))

In [5]:
common_drugs = pd.read_csv('../data/common_drugs.csv', sep=';')
common_drugs_ids = common_drugs.DrugBank_id.values
common_drugs_ids

array(['DB01558', 'DB08813', 'DB04817', 'DB00343', 'DB00829', 'DB00675',
       'DB00316', 'DB04817', 'DB00996', 'DB01009', 'DB01050', 'DB01373',
       'DB00451', 'DB00099'], dtype=object)

In [13]:
drug_id = common_drugs_ids[1]

predictions = pd.read_csv(prediction_dir + 'complex_' + drug_id + '_negative_pos_neg.csv', sep=',', index_col=[0])

# keep just drug/food/food compound predictions
predictions['node_type'] = list(itertools.repeat('xxx', predictions.shape[0]))
predictions.loc[predictions['tail_label'].str.contains("DB\d+", regex=True), 'node_type'] = "drug"
predictions.loc[predictions['tail_label'].str.contains("FDB"), 'node_type'] = "food_compound"
predictions.loc[predictions['tail_label'].str.contains("FOOD"), 'node_type'] = "food"
predictions = predictions.loc[predictions['node_type'] != 'xxx']
predictions

,tail_id,score,tail_label,in_validation,in_testing,node_type
3167,3167,9.967967,DB01268,False,False,drug
2819,2819,8.713588,DB00903,False,False,drug
2565,2565,8.559863,DB00635,False,False,drug
2460,2460,8.542276,DB00526,False,False,drug
3946,3946,8.483827,DB09220,False,False,drug
2356,2356,8.337879,DB00414,False,False,drug
3604,3604,8.130182,DB06695,False,False,drug
2192,2192,8.047192,DB00246,False,False,drug
4061,4061,7.909979,DB09477,False,False,drug
4025,4025,7.786120,DB09371,False,False,drug


In [14]:
# assign entity names to ids
for row in predictions.iterrows():
    tail = row[1].tail_label
    node_type = row[1].node_type
    
    if node_type == 'drug':
        tail_name = drug_id_map_dict[tail]
    elif node_type == 'food':
        tail_name = food_id_map_dict[tail]
    else:
        tail_name = compound_id_map_dict[tail]
        
    print(tail, tail_name)    

DB01268 Sunitinib
DB00903 Etacrynic acid
DB00635 Prednisone
DB00526 Oxaliplatin
DB09220 Nicorandil
DB00414 Acetohexamide
DB06695 Dabigatran etexilate
DB00246 Ziprasidone
DB09477 Enalaprilat
DB09371 Norethynodrel
DB00876 Eprosartan
DB11362 Selexipag
DB00457 Prazosin
DB00945 Acetylsalicylic acid
DB09125 Potassium citrate
DB01234 Dexamethasone
DB00870 Suprofen
DB08906 Fluticasone furoate
DB00310 Chlorthalidone
DB00177 Valsartan


In [9]:
# check if the predicted drug/food is in different interaction with the same drug in the training data

idx = drug_id

train_triplets = pd.read_csv(triplets_dir + 'train.tsv', sep='\t', index_col=[0])
filtered_triplets = train_triplets.loc[train_triplets.index == idx]
filtered_triplets.loc[filtered_triplets['tail'] == 'DB00903']

,relation,tail
head,,
